In [1]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import root_mean_squared_error
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
def read_dataframe(filename, 
                   drop_ouliers=False):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_pickup_datetime  = pd.to_datetime(df.tpep_pickup_datetime )
        df.tpep_dropoff_datetime   = pd.to_datetime(df.tpep_dropoff_datetime  )
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    if drop_ouliers:
        df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

## Q1. Downloading the data
We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".\
Download the data for January and February 2023.

Read the data for January. How many columns are there?

16\
17\
18\
19*

In [3]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
original_size_train = df_train.shape[0]
original_size_val = df_val.shape[0]

In [4]:
df_train.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.000,0.970,1.000,N,161,141,2,9.300,1.000,0.500,0.000,0.000,1.000,14.300,2.500,0.000,8.433
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.000,1.100,1.000,N,43,237,1,7.900,1.000,0.500,4.000,0.000,1.000,16.900,2.500,0.000,6.317
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.000,2.510,1.000,N,48,238,1,14.900,1.000,0.500,15.000,0.000,1.000,34.900,2.500,0.000,12.750
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.000,1.900,1.000,N,138,7,1,12.100,7.250,0.500,0.000,0.000,1.000,20.850,0.000,1.250,9.617
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.000,1.430,1.000,N,107,79,1,11.400,1.000,0.500,3.280,0.000,1.000,19.680,2.500,0.000,10.833


In [5]:
df_train.shape

(3066766, 20)

## Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

32.59\
42.59*\
52.59\
62.59


In [6]:
df_train.duration.describe()

count   3066766.000
mean         15.669
std          42.594
min         -29.200
25%           7.117
50%          11.517
75%          18.300
max       10029.183
Name: duration, dtype: float64

## Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

90%\
92%\
95%\
98%*

In [7]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet', drop_ouliers=True)
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet', drop_ouliers=True)
size_train = df_train.shape[0]
size_val = df_val.shape[0]

In [8]:
print(f'''
Fraction left after dropped the outliers
Train: {round(size_train*100/original_size_train)}%
Val: {round(size_val*100/original_size_val)}%
''')


Fraction left after dropped the outliers
Train: 98%
Val: 98%



## Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)\
Fit a dictionary vectorizer\
Get a feature matrix from it\
What's the dimensionality of this matrix (number of columns)?\

2\
155\
345\
515*\
715

In [9]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [14]:
categorical = ['PULocationID', 'DOLocationID'] #'PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [19]:
X_train.shape

(3009173, 515)

## Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters, where duration is the response variable\
Calculate the RMSE of the model on the training data\
What's the RMSE on train?

3.64\
7.64*\
11.64\
16.64

In [15]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)
root_mean_squared_error(y_train, y_pred)

np.float64(7.649261934850555)

## Q6. Evaluating the model
Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

3.81\
7.81*\
11.81\
16.81

In [17]:
y_pred = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred)

np.float64(7.811817745843695)